In [ ]:
import numpy as np

def projected_newton_eigen(A, x0, max_iter=50, tol=1e-10):
    """
    Newton/PINVIT style eigenvector solver with projection P(A - λI)P.
    Uses column vector form so xk @ xk.T is valid.
    """
    n = A.shape[0]
    
    # Ensure x0 is a column vector (n,1)
    xk = np.array(x0, dtype=float).reshape(n, 1)
    xk /= np.linalg.norm(xk)
    
    # Initial Rayleigh quotient
    lambdak = (xk.T @ A @ xk)[0,0]
    
    for k in range(max_iter):
        # Residual
        rk = A @ xk - lambdak * xk
        
        # Check convergence
        if np.linalg.norm(rk) < tol:
            break
        
        # Projector onto xk⊥
        P = np.eye(n) - (xk @ xk.T)
        
        # Build projected operator
        M = P @ (A - lambdak * np.eye(n)) @ P
        
        # Solve for δx in xk⊥
        try:
            delta_x = np.linalg.solve(M, -rk)
        except np.linalg.LinAlgError:
            raise RuntimeError("Projected operator became singular.")
        
        # Enforce orthogonality explicitly (numerical safeguard)
        delta_x -= xk * (xk.T @ delta_x)
        
        # Update and normalize
        xk = xk + delta_x
        xk /= np.linalg.norm(xk)
        
        # Update eigenvalue
        lambdak = (xk.T @ A @ xk)[0,0]
    
    return lambdak, xk

# ----------------------------
# Example: Test on 2×2 matrix
# ----------------------------
# Construct the 10x10 tridiagonal matrix with eigenvalues 1, 2, ..., 10
n = 10
main_diag = np.arange(1, n+1)  # Diagonal entries: 1, 2, ..., 10
off_diag = np.ones(n-1)         # Off-diagonal entries: 1, 1, ..., 1 (length 9)

# Create the tridiagonal matrix
A = np.diag(main_diag) + np.diag(off_diag, k=1) + np.diag(off_diag, k=-1)
x0 = np.random.rand(n)  # initial guess
lam, x = projected_newton_eigen(A, x0)

print("Approximate eigenvalue:", lam)
print("Approximate eigenvector:", x)
print("Residual norm:", np.linalg.norm(A @ x - lam * x))

'''
# compared with numpy's eigenvalue solver
eig_idx = n  # We want the nth eigenvalue (largest)
eigvals, eigvecs = np.linalg.eig(A)
print("Numpy Eigenvalues:", eigvals[eig_idx-1])
print("Numpy Eigenvectors:\n", eigvecs[:, eig_idx-1])
print("Numpy Residual norm:", np.linalg.norm(A @ eigvecs[:, eig_idx-1] - eigvals[eig_idx-1] * eigvecs[:, eig_idx-1]))
'''

Approximate eigenvalue: 7.003951798616377
Approximate eigenvector: [[ 0.00116964]
 [ 0.00702249]
 [ 0.03397053]
 [ 0.12899389]
 [ 0.3535209 ]
 [ 0.57944495]
 [ 0.2282139 ]
 [-0.5785431 ]
 [ 0.34804291]
 [-0.11616733]]
Residual norm: 2.651248324005928e-15
Numpy Eigenvalues: 10.746194182903322
Numpy Eigenvectors:
 [6.14228345e-07 5.98638873e-06 5.17438899e-05 3.94831830e-04
 2.61186831e-03 1.46134706e-02 6.67465010e-02 2.35431883e-01
 5.79795167e-01 7.77003065e-01]
Numpy Residual norm: 1.213108893411858e-15
